<a href="https://colab.research.google.com/github/fatemehes75/SQL_Challenge/blob/main/Datacenter_Server_Efficiency_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The platform develops a service that runs on a cluster of N servers in a single datacenter. Every request goes through load balancer that
sends this request to one of n instances for processing. Every server
in the cluster permanently collects and stores various internal
information for the last 24 hours.

* Developed a data analysis pipeline to process and analyze server performance data from a cluster of N servers in a single datacenter.

* Utilized SQL and Pandas to extract valuable insights, including identifying high-efficiency servers, uploader servers, and inefficient servers based on various performance metrics.

In [ ]:
# @title
%load_ext sql
import csv
import sqlite3
import pandas as pd

In [ ]:
# Load data from CSV into a Pandas DataFrame
df = pd.read_csv("data1.csv")
df.head()

,id,instance_name,active_time_sec,boot_time_sec,get_requests,post_requests,traffic_in_mb,traffic_out_mb,average_response_time_ms
0,0,inst-000,73564,218,8988,9089,7534,99576,154
1,1,inst-001,85054,120,7086,16439,9958,66000,169
2,2,inst-002,83930,326,16533,2541,19228,145098,196
3,3,inst-003,20731,232,7821,12759,3208,23049,119
4,4,inst-004,76530,2006,6404,6084,10843,172167,95


In [97]:
df['boot_time_percentage'] = (df['boot_time_sec'] / df['active_time_sec'])
df['total_requests'] = (df['get_requests'] + df['post_requests'])
df['in>out'] = df['traffic_in_mb'] > df['traffic_out_mb']
df.head()

,id,instance_name,active_time_sec,boot_time_sec,get_requests,post_requests,traffic_in_mb,traffic_out_mb,average_response_time_ms,boot_time_percentage,total_requests,in>out
0,0,inst-000,73564,218,8988,9089,7534,99576,154,0.002963,18077,False
1,1,inst-001,85054,120,7086,16439,9958,66000,169,0.001411,23525,False
2,2,inst-002,83930,326,16533,2541,19228,145098,196,0.003884,19074,False
3,3,inst-003,20731,232,7821,12759,3208,23049,119,0.011191,20580,False
4,4,inst-004,76530,2006,6404,6084,10843,172167,95,0.026212,12488,False


For example:
During the last 24 hours, server inst-000 was actively in work for
**73564** seconds. It has also rebooted one or more time, which took **218**
seconds in total. Note that booting time is not counted as active time
work. The server has processed **8988** GET requests and **9089**
POST requests. According to traffic measures, the incoming traffic is
7534 Mb and the outgoing traffic is 99756 Mb. The average instance response time is **154**.

In [ ]:
# Connect to SQLite database
con = sqlite3.connect("my_data1.db")
cur = con.cursor()
%sql sqlite:///my_data1.db

In [98]:
# Replace or create a new table "DATATBL" with the data from the DataFrame
df.to_sql("DATATBL", con, if_exists='replace', index=False)

300

In [ ]:
%sql SELECT * FROM DATATBL

Q1: The server is "high efficient"if it has a **response time under 100 ms**, **processed at least 22000 total
requests**, and has a **boot time percentage under 1% of active time**.
Count the number of high-efficient servers.

In [100]:
%sql SELECT Count(*) AS high_efficient_servers FROM DATATBL WHERE average_response_time_ms < 100 AND \
get_requests + post_requests >= 22000 AND boot_time_percentage < 0.01;

 * sqlite:///my_data1.db
Done.


high_efficient_servers
17


Q2: We mark the server to be "uploader" if the **incoming traffic is higher
than outgoing traffic** and the server has processed **at least 1000
POST requests**. Count the number of such servers.

In [101]:
%sql SELECT Count(*) AS uploader FROM DATATBL WHERE traffic_in_mb > traffic_out_mb	AND post_requests >= 1000;

 * sqlite:///my_data1.db
Done.


uploader
13


Q3: The server is "inefficient", if it has a **boot time above 0.2% of active working time** and has a **response time greater than 200 ms** and has **less than 5 total active work hours**, What percentage of servers are considered to be inefficient??

In [102]:
%sql SELECT Count(*) FROM DATATBL  \
WHERE average_response_time_ms > 200\
AND active_time_sec < 18000\
AND boot_time_percentage> 0.002;

 * sqlite:///my_data1.db
Done.


Count(*)
3


Q4: The server is "high efficient"if it has **a response time under 100 ms**, **processed at least 22000 total requests**, and has a **boot time percentage under 1% of active time**. And we mark the server to be "uploader" if the **incoming traffic is higher
than outgoing traffic** and the server has processed **at least 1000
POST requests**. Count the number of such servers that are **not efficient** and "uploader".

In [120]:
%sql SELECT Count(*) FROM DATATBL WHERE traffic_in_mb > traffic_out_mb	AND post_requests >= 1000\
AND (average_response_time_ms >= 100) AND (total_requests >= 22000) AND (boot_time_percentage <0.01)

 * sqlite:///my_data1.db
Done.


Count(*)
4


Q5: Percent of boot time can be calculated as the boot time of a server over its total active work time. For example, for a server with a boot time of 5 sec and active work time 500 sec, boot time takes 18 of active work time of the instance. What is the average percent of boot time over the servers?

In [104]:
%sql SELECT AVG (boot_time_percentage)*100 AS average_boot_time_percentage FROM DATATBL

 * sqlite:///my_data1.db
Done.


average_boot_time_percentage
0.651996918126827


Q6: Which percentage of servers has **response time > 0.2 s**?

In [118]:
%sql SELECT COUNT(*) AS percentage FROM DATATBL WHERE average_response_time_ms > 200

 * sqlite:///my_data1.db
Done.


percentage
44


In [119]:
(44/300)*100

14.666666666666666

In [ ]:
%sql SELECT video_id, path, duration FROM videosTBL ORDER BY duration DESC LIMIT 1;

 * sqlite:///my_data1.db
Done.


video_id,path,duration
2067,/videos/2020-02-04/nutation-centripetalisms.mp4,60


In [ ]:
%sql SELECT COUNT(*) FROM ads_statisticsTBL  AS asp LEFT JOIN platformsTBL AS p ON asp.platform_id=p.platform_id WHERE p.platform_id IS NULL;

 * sqlite:///my_data1.db
Done.


COUNT(*)
23
